# Problem Session 7
## Forecasting The Bachelorette and Pumpkin Spice II

In the second of two time series based problem sessions you build upon your work in `Problem Session 6`. In particular you will look to build the best forecast you can for the Bachelorette IMDB ratings. Afterwards you will practice using a SARIMA model with the pumpkin spice Google trends data.

The problems in this notebook will cover the content covered in our `Time Series Forecasting` lectures including:
- `Averaging and Smoothing`,
- `Stationarity and Autocorrelation` and
- `SARIMA`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import set_style
from datetime import datetime

set_style("whitegrid")

#### 1. The Bachelorette

##### a.
Here we:

- Reload the Bachelorette IMDB data stored in `the_bachelorette.csv` in the `data` folder. 
- Look at the first five rows.
- Make a train test split setting aside the last three episodes as a test set.
- Visualize the training set.

In [ ]:
tv = pd.read_csv(filepath_or_buffer="../../data/the_bachelorette.csv")
tv.head()

In [ ]:
tv_train = # your code here
tv_test = # your code here

In [ ]:
plt.figure(figsize=(16,5))

for season in range(1, tv_train.season.max()+1):
    plt.plot(tv_train.loc[tv_train.season==season].episode_number,
                tv_train.loc[tv_train.season==season].imdb_rating,
                '-o')

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.xlabel("Episode Number", fontsize=12)
plt.ylabel("IMDB Rating", fontsize=12)

plt.title("The Bachelorette IMDB Ratings\nColored by Season", fontsize=14)
    
plt.show()

Some notes on the dataset

* These are *subjective* assessments of episode quality which are made by self-selected raters.  This is far from a random sample.
* A lot of the variability will be due to things which are difficult to predict, such as popularity of particular contestants.

For these reasons we cannot expect to generate a forecast with much predictive ability.  We shouldn't really *expect* to be able to beat simple baselines like the naive baseline, rolling average, or random walk with drift.  The use of more "advanced" models is probably ill advised.  We are implementing more advanced models here to practice using them, not because it would be a good idea to actually trust these models.

##### b.

Here is a refresher on the columns of this data.

- `episode_number` is the number of the episode with respect to the entire series run,
- `title` is the title of the episode,
- `season` is the number of the season in which the episode aired,
- `season_episode_number` is the number of the episode with respect to the season in which it aired,
- `imdb_rating` is the average rating of the episode among IMDB's users.

##### c.

The first model you will fit is a rolling average model. In this problem you will be tuning the moving average window size, $q$, to find the value that minimizes the Mean Absolute Scaled Error (defined below).

Fill in the missing chunks of code to perform hyperparameter tuning for $q$.

In [ ]:
# This function takes 3 numpy arrays as inputs.
# It returns the 
# mean absolute error of the forecast on the test set 
# relative to the
# mean absolute error of the naive forecast on the training set over an equivalent horizon.
# 
# If MASE > 1 then your forecast performs worse out of sample than naive forecast does in sample.
# If MASE < 1 then your forecast performs better out of sample than naive forecast does in sample.

def mase(y_train, y_test, y_preds):
    n = len(y_train)
    m = len(y_test)
    denom = 0
    for i in range(n-m):
        denom += np.abs(y_train[i+1:i+m+1] - y_train[i]*np.ones(m)).mean()
    denom = denom / (n-m)
    num = np.abs(y_test - y_preds).mean()
    return num/denom

# Example calculation

print('MASE = ', mase(y_train = np.array([1,2,3,4,5]), y_test = np.array([6,7]), y_preds = np.array([3, 3])))


This would be computed by hand as follows:

$$
\frac{\textrm{Mean absolute prediction error out of sample}}{\textrm{Mean absolute naive forecast prediction error in sample}} = \frac{\frac{1}{2} \left( |6-3| + |7-3|\right)}{\frac{1}{3} \left[ \frac{1}{2}\left(|2-1| + |3-1| \right) + \frac{1}{2}\left(|3-2| + |4-2| \right) + \frac{1}{2}\left(|4-3| + |5-3| \right)  \right]} = 2.333
$$


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
cv = TimeSeriesSplit(10, test_size=3)

start = 2
end = 31
ra_mase = np.zeros((10, len(range(start, end))))


i = 0
for train_index, test_index in cv.split(tv_train):
    tv_tt = # your code here
    tv_ho = # your code here
    
    j = 0
    for q in range(start, end):
        pred = # your code here
        
        ra_mase[i,j] = # your code here
        j = j + 1
    i = i + 1

In [ ]:
plt.figure(figsize=(12,5))

plt.scatter(range(start,end), np.mean(ra_mase, axis=0))

plt.xlabel("Window Size", fontsize=12)
plt.ylabel("Average MASE", fontsize=12)

plt.xticks(range(start, end, 3), fontsize=10)
plt.yticks(fontsize=10)

plt.show()

In [ ]:
print("The window size that minimized the avg. cv MASE",
      "was q =", 
      range(start,end)[np.argmin(np.mean(ra_mase, axis=0))],
      "\b.",
      "It had a mean cv MASE of", 
      np.round(np.min(np.mean(ra_mase, axis=0)), 3))

##### d.

The second model you will try is an exponential smoothing model.

Because these data exhibit a trend but not seasonality we will fit a double exponential smoothing model. For this we will want to find the best $\alpha$ (The smoothing on the time series) and $\beta$ (the smoothing on the trend component).

Fill in the missing code chunks below to perform a grid search for the values of $\alpha$ and $\beta$ that minimize the average CV RMSE. (Note that a grid search is what we call it when you perform hyperparameter tuning with a grid of possible hyperparameter values).

In [ ]:
from statsmodels.tsa.holtwinters import Holt

In [ ]:
exp_mase = np.zeros((10, len(np.arange(0, 0.2, .01)), len(np.arange(0, 0.2, .01))))

i = 0
for train_index, test_index in cv.split(tv_train):
    tv_tt = # your code here
    tv_ho = # your code here
    
    j = 0
    for alpha in np.arange(0, 0.2, .01):
        k = 0
        for beta in np.arange(0, 0.2, .01):
            exp_smooth = # your code here

            exp_mase[i,j,k] = # your code here
            k = k + 1
        j = j + 1
    i = i + 1

In [ ]:
## This gives us the indices of the smallest
## avg cv rmse
exp_ind = np.unravel_index(np.argmin(np.mean(exp_mase, axis=0), axis=None), 
                           np.mean(exp_mase, axis=0).shape)
np.unravel_index(np.argmin(np.mean(exp_mase, axis=0), axis=None), 
                 np.mean(exp_mase, axis=0).shape)

In [ ]:
print("The alpha and beta values that give a double exponential",
         "smoothing model with lowest avg cv rmse are",
         "alpha = ", np.arange(0, 0.2, .01)[exp_ind[0]],
         "and beta = ", np.arange(0, 0.2, .01)[exp_ind[1]])

print("This model had an avg cv rmse of",
         np.round(np.mean(exp_mase, axis=0)[exp_ind],3))

##### e.

The final model you will try is an ARIMA model. 

First let's check the stationarity assumption for this time series. Make an autocorrelation plot of the training data. If you find that the ACF plot indicates that the time series is non-stationary, plot the ACF of the time series' first differences. Do these appear to be stationary?

In [ ]:
import statsmodels.api as sm

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,5))

sm.graphics.tsa.plot_acf(tv_train.imdb_rating.values,
                            lags = 40,
                            ax = ax)

plt.title('The Bachelorette IMDB rating ACF', fontsize=14)
plt.ylabel("Autocorrelation", fontsize=12)
plt.xlabel("Lag", fontsize=12)

plt.ylim(-1.1,1.1)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))

# plot ACF of first differenced data here


plt.title('The Bachelorette First Differences ACF', fontsize=14)
plt.ylabel("Autocorrelation", fontsize=12)
plt.xlabel("Lag", fontsize=12)

plt.ylim(-1.1,1.1)

plt.show()

##### f.

From what we saw above what we should set our $d$ value in the ARIMA model?. Set $d$ to this value and then perform hyperparameter tuning to find the values of $p$ and $q$ that give us the lowest mean CV RMSE.

In [ ]:
from statsmodels.tsa.api import ARIMA

In [ ]:
arima_mase = np.zeros((10, 4, 4))

i = 0
for train_index, test_index in cv.split(tv_train):

    tv_tt = tv_train.iloc[train_index]
    tv_ho = tv_train.iloc[test_index]
    
    j = 0
    for p in range(4):
        k = 0
        for q in range(4):
            arima = # your code here
            
            arima_mase[i,j,k] = # your code here
            k = k +1
        j = j + 1
    i = i +1

In [ ]:
arima_ind = np.unravel_index(np.argmin(np.mean(arima_mase, axis=0), axis=None), 
                             np.mean(arima_mase, axis=0).shape)
np.unravel_index(np.argmin(np.mean(arima_mase, axis=0), axis=None), 
                 np.mean(arima_mase, axis=0).shape)

In [ ]:
print("The parameters that give an ARIMA model",
         "with lowest avg cv mase are",
         "(p,d,q) = ( %s, %s, %s )" %(range(4)[arima_ind[0]], 1, range(4)[arima_ind[1]]))

print("This model had an avg cv mase of",
         np.round(np.mean(arima_mase, axis=0)[arima_ind],3))

As an alternative to cross validation MASE, another common way to select ARIMA parameters is by minimizing the [Akaike Information Criterion (AIC)](https://en.wikipedia.org/wiki/Akaike_information_criterion) on the training set.  Let's see whether this yields the same hyperparameters as we obtained above.  There is a handy object called `auto_arima` from `pmdarima` which can do automatic order selection in this way.

In [ ]:
from pmdarima import auto_arima

In [ ]:
auto_arima(tv_train.imdb_rating.values, trace=True, max_p=4, max_q=4)

We obtained the same parameters!  Let's also check the model summary.

In [ ]:
arima = ARIMA(tv_train.imdb_rating.values,order = # your code here).fit()
arima.summary()

##### g.

Plot your forecast (along with the training and test data) using the model with the lowest CV MASE.

What is the MASE on the test set?

In [ ]:
test_mase = # your code here

In [ ]:
plt.figure(figsize=(14,5))

plt.scatter(tv_train.episode_number,
               tv_train.imdb_rating,
               alpha=.5,
               label="Training Points")

plt.scatter(tv_test.episode_number,
               tv_test.imdb_rating,
               alpha=.5,
               c = 'red',
               marker = 'v',
               label="Test Points")

plt.plot(tv_train.episode_number,
            # your code here,
            'k-',
            linewidth = 2,
            label="Fitted Values")

plt.plot(tv_test.episode_number,
            # your code here,
            'r--',
            linewidth=2,
            label="Forecast")

plt.legend(fontsize=12, loc=3)

plt.title("Test Set MASE = " + str(np.round(test_mase,3)),
             fontsize=14)

plt.xlabel("Episode Number", fontsize=12)
plt.ylabel("IMDB Rating", fontsize=12)


plt.ylim(3,8.5)
plt.xlim(180,220)

plt.show()

#### 2. Pumpkin spice SARIMA

We won't have enough time in this problem session to do more extensive model building and comparison for the Pumpkin Spice data forecasting problem.  We will just use this as opportunity to practice fitting SARIMA models.

Load the data stored in `pumpkin_spice.csv` in the `Data` folder then look at the first five rows. Then make a train test split setting aside all observations on or after January 1, 2022 aside as the test set.

In [ ]:
pumpkin = pd.read_csv("../../data/pumpkin_spice.csv",
                         parse_dates = ["Month"])

In [ ]:
p_train = pumpkin.loc[pumpkin.Month < datetime(2022, 1, 1)].copy()
p_test = pumpkin.drop(p_train.index).copy()

In [ ]:
plt.figure(figsize=(16,5))

plt.plot(pumpkin.Month,
            pumpkin.interest_level,
            '-o')

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.xlabel("Date", fontsize=12)
plt.ylabel("Google Interest Level", fontsize=12)

plt.show()

##### b.

In lecture we talked about first differencing non-stationary time series exhibiting a trend to create a, seemingly, stationary time series.

This can also be done for seasonal data. Suppose that we suspect a time series, $\left\lbrace y_t \right\rbrace$ exhibits seasonality where a season lasts $m$ time steps. Then the first seasonal differenced time series is:

$$
\nabla_s y_t = y_t - y_{t-m}.
$$

Plot the autocorrelation of the training set, then perform first seasonal differencing on these data and plot the autocorrelation of the first seasonal differenced series.

Does the differenced series appear less likely to violate stationarity?

In [ ]:
import statsmodels.tsa.api as sm

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))

sm.graphics.plot_acf(p_train.interest_level.values,
                        lags = 36,
                        ax = ax)

plt.title('Autocorrelation of the original data')

plt.ylim([-1.1,1.1])

plt.show()

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))

# plot first seasonal differenced data here

plt.title('Autocorrelation of first seasonal differenced data')
plt.ylim([-1.1,1.1])

plt.show()

##### c.

Recall for an $\text{ARIMA}$ model you needed parameters $p$, $d$ and $q$. For a $\text{SARIMA}$ model you need parameters $P$, $D$, $Q$ and $m$ as well:

- $P$ is the order of the seasonal autoregressive portion of the model,
- $Q$ is the order of the seasonal moving average portion of the model,
- $D$ is the order of the seasonal differencing and
- $m$ is the number of time steps that take place in a single period.

Use `auto_arima` with $D=1$ and $m=12$ and determine the AIC minimizing values of $p,d,q,P,Q$.

In [ ]:
# use auto_arima to identify the best hyperparameters here 

In [ ]:
#fit an ARIMA model using those hyperparameters

sarima = # your code here

In [ ]:
plt.figure(figsize=(14,6))

plt.plot(p_train.Month,
            p_train.interest_level,
            'b-o',
            label='Training Data')


plt.plot(p_test.Month,
            p_test.interest_level,
            color = 'orange',
            marker = 'o',
            label='Test Data')

plt.plot(p_train.Month,
            sarima.fittedvalues,
            'r',
            label='Fitted Values')

plt.plot(p_test.Month,
               sarima.forecast(len(p_test)),
               '--r',
               label="Forecast")


plt.legend(fontsize=14)
plt.xlabel("Date", fontsize=14)
plt.ylabel("Interest Level", fontsize=14)

plt.show()

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023. Modified by Steven Gubkin 2024.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)